##Extract Data

In [2]:
import numpy as np
from sklearn import preprocessing
import tensorflow as tf
import tensorflow_datasets as tfds

#Fetch dataset directly
mnist_dataset, mnist_info = tfds.load(name='rock_paper_scissors', with_info=True, as_supervised=True)
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

##Split the Dataset into Train, Validation, and Test

In [3]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    resized_image = tf.image.resize(image, (150, 150))
    return resized_image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)


BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 50

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

##Model Outline

In [38]:
input_size = 784
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(150, 150, 3)),
        tf.keras.layers.Dense(hidden_layer_size, activation='elu'),
        tf.keras.layers.Dense(output_size, activation='softmax')
        ])

model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUM_EPOCHS = 20

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)
model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets),
          validation_steps=10, verbose=2)

Epoch 1/20
46/46 - 4s - loss: 73.0896 - accuracy: 0.3527 - val_loss: 22.0967 - val_accuracy: 0.3532 - 4s/epoch - 97ms/step
Epoch 2/20
46/46 - 3s - loss: 43.3798 - accuracy: 0.3690 - val_loss: 61.6608 - val_accuracy: 0.3532 - 3s/epoch - 69ms/step
Epoch 3/20
46/46 - 5s - loss: 33.1810 - accuracy: 0.4339 - val_loss: 13.8955 - val_accuracy: 0.4563 - 5s/epoch - 109ms/step
Epoch 4/20
46/46 - 3s - loss: 25.0178 - accuracy: 0.5026 - val_loss: 46.3163 - val_accuracy: 0.5556 - 3s/epoch - 72ms/step
Epoch 5/20
46/46 - 4s - loss: 23.7436 - accuracy: 0.5066 - val_loss: 37.8037 - val_accuracy: 0.3889 - 4s/epoch - 80ms/step
Epoch 6/20
46/46 - 5s - loss: 17.0469 - accuracy: 0.5622 - val_loss: 5.9793 - val_accuracy: 0.5675 - 5s/epoch - 103ms/step
Epoch 7/20
46/46 - 4s - loss: 14.7568 - accuracy: 0.5891 - val_loss: 25.0148 - val_accuracy: 0.4127 - 4s/epoch - 80ms/step
Epoch 8/20
46/46 - 3s - loss: 14.9326 - accuracy: 0.5203 - val_loss: 4.7475 - val_accuracy: 0.6706 - 3s/epoch - 76ms/step
Epoch 9/20
46/46

##Test the Model

In [39]:
test_loss, test_accuracy = model.evaluate(test_data)
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

1/1 [==============================] - 0s 380ms/step - loss: 8.5994 - accuracy: 0.4919

Test loss: 8.60. Test accuracy: 49.19%
